In [143]:
import params
from google.cloud import datastore, storage, logging
import time
import pickle
import hashlib
import sys
import numpy as np
import portfolioGeneration
import portfolio
import dataAck
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")
import multiprocessing as mp 
import autoPortfolioTree
import curveTreeDB
import portfolio

In [144]:
possibleModels = [item["model"] for item in curveTreeDB.getValidModels(params.treeModels, returnEntireObject=True)]
allModels = []
tickersSeen = []
for i in range(len(possibleModels)):
    model = possibleModels[i]
    if model.numberOfPredictors() > 4:
        print(model.targetTicker, model.getHash(), model.numberOfPredictors())
        allModels.append(model)
        if model.targetTicker not in tickersSeen:
            tickersSeen.append(model.targetTicker)

EWJ d9c92dc0c1806652b36892901a9c606d38ecf87fcf0c55d207d0ef7e 5
USO a59c5f0cddc6a96c637a876c690b6b355883219c39a31d40d8ac9ded 5
DXJ a108a2ab55a80d8da2dd2b8d6de7bb6a472141282fe913cade1a60b0 6
XHB e58e88dc223e92a3c2fd0bffae405bcd61880f2efc6d774c825c8a09 5
ITB 7bf216084e1827363b8eb8987211719c40dfb929738fa8ac5220989f 5
USO 28b303f2cf4958bed1c4e4b5268bb0e52ccb7dc7d30ce1151aabd567 5
SLV 61447af990efe38facca483d84a141cd9f4508b2f8e922abe40c48b0 5
ITB 7962430c90a8864c374f73b381af75fec0f5b5a954cec3bcf6e2078f 5
USO 3af24709486c14cd9f02a217f7e289eb52a718df584b7ec47b813598 6
DXJ 6007373d9f2522f3b4c909f4e9b064514d1cce1b311d530826f5f9f2 5
XHB d5c7a8898641f5b6258a9a279d87d28915185c0e3a5c5a96980fb5da 6
USO 6ffced1563506524b477403efcb8232d65ae4590499e9afe321caf00 5
VFH f29905ddb84c82fe6d230afa0bde51110bbf8760e6891b65f7151c85 6
USO f72cc18346ae29af6240adf1ff99231d55f41abd614341fb003f29c3 5
GDX 291ae9de9091e72e76076033684f92ad5d0b9ce4dcefffb0b783eec4 5
USO 1f1fdaa78af0037a9f1312dff9f4772b95d3d17dad46f22450d

In [145]:
len(allModels)

31

In [147]:
tickersSeen

['EWJ', 'USO', 'DXJ', 'XHB', 'ITB', 'SLV', 'VFH', 'GDX', 'OIH', 'XBI']

In [148]:
import random
factorToTrade = tickersSeen[random.randint(0, len(tickersSeen) - 1)]
factorToTrade

'XHB'

In [150]:
uniqueModels, modelReturns, modelPredictions, modelSlippageReturns, modelReturnsWithFactor, joinedData = autoPortfolioTree.computeReturnsForUniqueModelsCache(allModels, factorToTrade)


['EWJ', 'USO', 'DXJ', 'XHB', 'ITB', 'SLV', 'VFH', 'GDX', 'OIH', 'XBI']
ITB
USO
SLV
GDX
DXJ
XHB
VFH
OIH
EWJ
XBI
ATTEMPTING PULL d9c92dc0c1806652b36892901a9c606d38ecf87fcf0c55d207d0ef7e
((('DVY', 36, 18, None, 1), 'EWJ', 10, 5, 0.7, 0.2, 10), (((('SOXX', 44, 10, 9, 2), 'EWJ', 10, 5, 0.5, 0.01, 10), (('IAU', 29, None, None, 5), 'EWJ', 5, 5, 0.5, 0.05, 10), 5, 'AND'), ((('IAU', 29, None, None, 5), 'EWJ', 5, 5, 1.0, 0.01, 10), (('IAU', 30, None, 18, 2), 'EWJ', 5, 5, 0.3, 0.1, 10), 5, 'AND'), 5, 'OR'), 5, 'OR')
ATTEMPTING PULL a59c5f0cddc6a96c637a876c690b6b355883219c39a31d40d8ac9ded
((('AGG', 35, 6, None, 1), 'USO', 44, 3, 2.0, 0.01, 10), (((('VBR', 37, None, None, 4), 'USO', 5, 3, 0.5, 0.05, 10), (('XLF', 15, 5, None, 1), 'USO', 10, 3, 0.7, 0.1, 10), 3, 'OR'), ((('VTV', 42, 8, None, 4), 'USO', 44, 3, 2.0, 0.01, 10), (('IWO', 21, None, None, 4), 'USO', 22, 3, 2.0, 0.05, 10), 3, 'OR'), 3, 'OR'), 3, 'OR')
ATTEMPTING PULL a108a2ab55a80d8da2dd2b8d6de7bb6a472141282fe913cade1a60b0
((((('XLB', 24, 

In [156]:
cleanedReturns = modelReturns.dropna()
cleanedReturns.columns = [item.getHash() for item in uniqueModels]

cleanedPredictions = modelPredictions.dropna()
cleanedPredictions.columns = [item.getHash() for item in uniqueModels]
hashToModel = {}
for item in uniqueModels:
    hashToModel[item.getHash()] = item

In [157]:
def historicalWeightsToTickerAllocations(historicalWeights, algorithmPredictions, modelsInPortfolio):
    aggregatePredictions = algorithmPredictions.dropna()
    allocationsToStore = []
    historicalAllocations = None
    scaledHistoricalAllocations = None
    ##ITERATE THROUGH DAYS TO CALCULATE NET POSITION
    for i in range(len(historicalWeights)):
        netPosition = {}
        weights = historicalWeights.iloc[i]
        for model in modelsInPortfolio:
            if model.targetTicker not in netPosition:
                netPosition[model.targetTicker] = 0.0
            try:
                aggregatePredictions.loc[historicalWeights.index[i]]
            except:
                continue
            
            netPosition[model.targetTicker] += weights[model.getHash()] * aggregatePredictions.loc[historicalWeights.index[i]][model.getHash()]
        thisDf = pd.DataFrame([netPosition], index=[historicalWeights.index[i]])
        if historicalAllocations is None:
            historicalAllocations = thisDf
        else:
            historicalAllocations = pd.concat([historicalAllocations, thisDf])
        
        totalCapitalUsed = sum([abs(netPosition[ticker]) for ticker in netPosition])
        scaledNetPosition = {}
        for ticker in netPosition:
            scaledNetPosition[ticker] = netPosition[ticker] * 1.0/totalCapitalUsed
        
        thisDf = pd.DataFrame([scaledNetPosition], index=[historicalWeights.index[i]])
        if scaledHistoricalAllocations is None:
            scaledHistoricalAllocations = thisDf
        else:
            scaledHistoricalAllocations = pd.concat([scaledHistoricalAllocations, thisDf])
    
    return historicalAllocations, scaledHistoricalAllocations
            
            

In [158]:
import empyrical
def getLimitedDataForPortfolio(historicalWeights, historicalPredictions, modelsUsed, factorToTrade, joinedData):
    
    normalTickerAllocationsTable, scaledTickerAllocationsTable = historicalWeightsToTickerAllocations(historicalWeights, historicalPredictions, modelsUsed)
    
    tickerAllocationsTable = scaledTickerAllocationsTable
    rawTickerPerformance = portfolioGeneration.calculatePerformanceForTable(tickerAllocationsTable, tickerAllocationsTable.columns, joinedData)

    rawAlgoPerformance = pd.DataFrame(rawTickerPerformance.apply(lambda x:sum(x), axis=1), columns=["Algo Return Without Commissions"])

    tickerPerformance, algoPerformance, algoTransactionCost =  portfolioGeneration.calculatePerformanceForAllocations(tickerAllocationsTable, joinedData)

    benchmark = factorToTrade
    factorReturn = dataAck.getDailyFactorReturn(benchmark, joinedData)
    factorReturn.columns = ["Factor Return (" + benchmark + ")"]
    algoPerformance.columns = ["Algo Return"]

    alpha, beta = empyrical.alpha_beta(algoPerformance, factorReturn)
    sharpe_difference = empyrical.sharpe_ratio(algoPerformance) - empyrical.sharpe_ratio(factorReturn)
    annualizedReturn = empyrical.annual_return(algoPerformance)[0]
    annualizedVolatility = empyrical.annual_volatility(algoPerformance)
    
    ##AUTOMATICALLY TAKES SLIPPAGE INTO ACCOUNT
    return {
        "benchmark":factorToTrade,
        "alpha":alpha,
        "beta":beta,
        "sharpe difference":sharpe_difference,
        "annualizedReturn":annualizedReturn,
        "annualizedVolatility":annualizedVolatility,
        "sharpe":empyrical.sharpe_ratio(algoPerformance),
        "free return":annualizedReturn - annualizedVolatility
    }
    


In [159]:
def returnSelectAlgos(algoColumns):
    return np.random.choice(algoColumns, size=random.randint(10, len(algoColumns)), replace= False)


In [160]:
import hrpPortfolioOpt as hrp
def produceHRPPredictions(aggregateReturns, windowSize, startIndex, maxWindowSize = False):
    hrpReturns = pd.DataFrame([])
    historicalWeights = pd.DataFrame([])
    i = windowSize
    if startIndex is not None:
        i = startIndex
    while i < len(aggregateReturns):
        corr = None
        cov = None
        if maxWindowSize == False:
            corr = (aggregateReturns[:i]).corr()
            cov = (aggregateReturns[:i]).cov()
        else:
            corr = (aggregateReturns[i-windowSize:i]).corr()
            cov = (aggregateReturns[i-windowSize:i]).cov()
        weights = hrp.getHRP(cov, corr)
    #     display(weights)
    #     display(aggregateReturns[i+windowSize:i+windowSize+1])
        todayReturn = aggregateReturns[i:i+1] * weights
    #     display(todayReturn)
        sumReturn = pd.DataFrame(todayReturn.apply(lambda x:sum(x), axis=1))
        hrpReturns = pd.concat([hrpReturns, sumReturn])
        thisWeights = pd.DataFrame([[weights[item] for item in weights.index]], index=sumReturn.index, columns=weights.index.tolist())
        historicalWeights = pd.concat([historicalWeights, thisWeights])
        i += 1
    return hrpReturns, historicalWeights

In [ ]:
def storeDiscoveredPortfolio(models, benchmark, IS_DATA, OOS_DATA):
    description = "AUTO GENERATED"
    portfolioType = "HRP FULL"
    allHashes = []
    for model in models:
        allHashes.append(model.getHash())
        
    ##SORT SO ENSURE SAME PORTFOLIO NOT CREATED TWICE
    allHashes = sorted(allHashes)
    
    portfolioString = str(allHashes) + benchmark + description + portfolioType
    portfolioHash = hashlib.sha224(portfolioString.encode('utf-8')).hexdigest()
    print("PORTFOLIO HASH:", portfolioHash)
    for hashing in allHashes:
        print(hashing)
        
    
        ##UPLOAD ORGANISM OBJECT
        while True:
            try:
                toUpload = {
                    "portfolio":portfolioHash,
                    "model":hashing
                }
                datastoreClient = datastore.Client('money-maker-1236')
                #HASH DIGEST
                key = datastoreClient.key(params.discoveredPortfolioModels, hashlib.sha224(str(hashing + portfolioHash).encode('utf-8')).hexdigest()) #NEED TO HASH TO ENSURE UNDER COUNT
                organismToStore = datastore.Entity(key=key)
                organismToStore.update(toUpload)
                datastoreClient.put(organismToStore)
                break
            except:
                print("UPLOAD ERROR:", str(sys.exc_info()))
                time.sleep(10)
    
    ##STORE PORTFOLIO OBJECT
    while True:
        try:
            toUpload = {
                "description":description,
                "benchmark":benchmark,
                "portfolioType":portfolioType,
                "startedTrading":curveTreeDB.getToday()
            }
            
            for k in IS_DATA:
                toUpload["IS_"+ k] = IS_DATA[k]
            
            for k in OOS_DATA:
                toUpload["OOS_"+ k] = OOS_DATA[k]
            
            datastoreClient = datastore.Client('money-maker-1236')
            #HASH DIGEST
            key = datastoreClient.key(params.discoveredPortfolios, portfolioHash) #NEED TO HASH TO ENSURE UNDER COUNT
            organismToStore = datastore.Entity(key=key)
            organismToStore.update(toUpload)
            datastoreClient.put(organismToStore)
            return portfolioHash
            break
        except:
            print("UPLOAD ERROR:", str(sys.exc_info()))
            time.sleep(10)

In [161]:
def performPortfolioPerformanceEstimation(historicalPredictions, historicalReturns, factorToTrade, hashToModel, joinedData):
    returnWindows = [(None, historicalReturns[:400]), (400, historicalReturns)]
    historicalWeights = None
    for selectedReturns in returnWindows:
        startIndex = selectedReturns[0]
        returnWindow = selectedReturns[1]
        hrpReturns, weightsSeen = produceHRPPredictions(returnWindow,\
                126, startIndex=startIndex, maxWindowSize=False)
        print("COMPUTED HISTORICAL WEIGHTS")
        if historicalWeights is None:
            historicalWeights = weightsSeen
        else:
            historicalWeights = pd.concat([historicalWeights, weightsSeen])
        
        modelsUsed = []

        tickersSeen = {}

        for modelHash in historicalPredictions.columns:
            thisModel = hashToModel[modelHash]
            modelsUsed.append(thisModel)
        if startIndex is None:
            scaledStats = getLimitedDataForPortfolio(historicalWeights,\
                                    historicalPredictions, modelsUsed, factorToTrade, joinedData)
            print(scaledStats)
            if scaledStats["sharpe difference"] < 0.0 or scaledStats["annualizedReturn"] < scaledStats["annualizedVolatility"]:
                return None, None
    trainStats = getLimitedDataForPortfolio(historicalWeights[:-252], \
                                              historicalPredictions, modelsUsed, factorToTrade, joinedData)
    testStats = getLimitedDataForPortfolio(historicalWeights[-252:], \
                                              historicalPredictions, modelsUsed, factorToTrade, joinedData)
    
    if trainStats["sharpe difference"] > 0.0 and trainStats["annualizedReturn"] > trainStats["annualizedVolatility"]:
        print(trainStats, testStats)
        storeDiscoveredPortfolio(modelsUsed, trainStats, testStats)
    

In [163]:
## MP RUN            
def createPossiblePortfoliosMP(cleanedPredictions, cleanedReturns, hashToModel, joinedData, threadsToUse):
    mpEngine = mp.get_context('fork')
        
    runningP = []
    while True:
        selectedAlgorithms = returnSelectAlgos(cleanedReturns.columns)
        print(len(selectedAlgorithms))
        factorToTrade = hashToModel[selectedAlgorithms[random.randint(0, len(selectedAlgorithms) - 1)]].targetTicker
        print(factorToTrade)
        while len(runningP) > threadsToUse:
            runningP = dataAck.cycleP(runningP)
        
        p = mpEngine.Process(target=performPortfolioPerformanceEstimation, args=(cleanedPredictions[selectedAlgorithms],\
                    cleanedReturns[selectedAlgorithms], factorToTrade, hashToModel, joinedData))
        p.start()
        runningP.append(p)

# In[ ]:


In [164]:
print("STARTING GENERATION")

##REMOVE BREAK TO DO FULL AUTO
createPossiblePortfoliosMP(cleanedPredictions, cleanedReturns, hashToModel, joinedData, threadsToUse=0)

STARTING GENERATION
21
XHB
24
ITB
COMPUTED HISTORICAL WEIGHTS
{'benchmark': 'XHB', 'alpha': 0.0648249539358521, 'beta': 0.018388947856885399, 'sharpe difference': 2.5263597049864885, 'annualizedReturn': 0.067285208111367734, 'annualizedVolatility': 0.02438993405029084, 'sharpe': 2.6823617736370462} {'benchmark': 'XHB', 'alpha': 0.4463321282277181, 'beta': 0.087266889674225051, 'sharpe difference': 2.1676995768951164, 'annualizedReturn': 0.53774648279405679, 'annualizedVolatility': 0.19329867710974166, 'sharpe': 2.323701645545674}
COMPUTED HISTORICAL WEIGHTS
train ({'benchmark': 'XHB', 'alpha': 0.06513660139910891, 'beta': 0.016070976544025208, 'sharpe difference': 2.4665639303882205, 'annualizedReturn': 0.06769437300722414, 'annualizedVolatility': 0.025098765730737244, 'sharpe': 2.6225659990387782}, {'benchmark': 'XHB', 'alpha': 0.4366631000089456, 'beta': 0.062340481134984096, 'sharpe difference': 2.0303442154046687, 'annualizedReturn': 0.52049811301836613, 'annualizedVolatility': 0.2

Process ForkProcess-377:
  File "/Users/patrickogrady/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py", line 1325, in __getitem__
    return self._getitem_tuple(key)
Traceback (most recent call last):
  File "/Users/patrickogrady/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/patrickogrady/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-161-44b81b9db6d6>", line 7, in performPortfolioPerformanceEstimation
    hrpReturns, weightsSeen = produceHRPPredictions(returnWindow,                126, startIndex=startIndex, maxWindowSize=False)
  File "<ipython-input-160-bdd8563466f0>", line 17, in produceHRPPredictions
    weights = hrp.getHRP(cov, corr)
  File "/Users/patrickogrady/Documents/CODE/walkforwardTrader/hrpPortfolioOpt.py", line 98, in getHRP
    hrp = getRecBipart(cov, sortIx)
  File "/Users/patrickogrady/Documents/CODE/wal

KeyboardInterrupt: 